<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>ChatGPT and Friends</h1>
<h1>GPT Models</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import openai
from openai import OpenAI
import termcolor
from termcolor import colored

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.21.0

Compiler    : Clang 15.0.0 (clang-1500.1.0.2.5)
OS          : Darwin
Release     : 23.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 16
Architecture: 64bit

Git hash: 2305f34fce93e0f71d7791f555117afb877e0f0c

openai    : 1.12.0
tqdm      : 4.66.1
numpy     : 1.26.4
json      : 2.0.9
termcolor : 2.4.0
watermark : 2.4.3
matplotlib: 3.8.2
pandas    : 2.2.0



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Basic Usage

The first step is always to load up the API key from the local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

In [4]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

We start by getting a list of supported models.

In [5]:
model_list = client.models.list().data

In total we have 27 models

In [6]:
len(model_list)

27

Along with some information about each model...

In [7]:
model_list[:3]

[Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
 Model(id='dall-e-2', created=1698798177, object='model', owned_by='system')]

But let's just get a list of model names

In [8]:
print("\n".join(sorted([model.id for model in model_list])))

babbage-002
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-0301
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo-preview
gpt-4-vision-preview
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


## Basic Prompt

The recommended model for exploration is `gpt-3.5-turbo`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [9]:
%%time
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

CPU times: user 6.95 ms, sys: 1.85 ms, total: 8.8 ms
Wall time: 798 ms


Which produces a response object

In [10]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

Which we can treat as a simple object

In [11]:
pprint(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's weakness is kryptonite, a mineral originating from his home planet of Krypton. Exposure to kryptonite weakens Superman and can even have fatal effects on him.", role='assistant', function_call=None, tool_calls=None))


The model answer can be found in the "message" dictionary inside the "choices" list

In [12]:
response.choices[0].message.content

"Superman's weakness is kryptonite, a mineral originating from his home planet of Krypton. Exposure to kryptonite weakens Superman and can even have fatal effects on him."

To request multiple answers, we must include the `n` parameter with the number of answers we want

In [13]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "What are the different kinds of Kryptonite?"},
    ],
    n=3
)

CPU times: user 6.49 ms, sys: 1.79 ms, total: 8.28 ms
Wall time: 5.03 s


And we can access each of the answers individually int he choices list

In [14]:
for output in response.choices:
    print("==========")
    print(output.message.role.title()) 
    print("==========")
    print(output.message.content)
    print("==========\n")

Assistant
In the DC Comics universe, there are various types of Kryptonite, each having different effects on Superman and other Kryptonians. Some of the different kinds of Kryptonite include:

1. Green Kryptonite: This is the most common form of Kryptonite and is deadly to Superman. It weakens, sickens, and can eventually kill him if he is exposed to it for a prolonged period. It is usually depicted as a green crystalline substance.

2. Red Kryptonite: Red Kryptonite has unpredictable effects on Superman, typically leading to temporary, random, and bizarre changes in his behavior or physical attributes. The effects are temporary and can vary from story to story.

3. Blue Kryptonite: Blue Kryptonite has a unique effect on Bizarro, a flawed and imperfect clone of Superman. It weakens and can even kill Bizarro, acting as an opposite counterpart to Green Kryptonite's effect on Superman.

4. Gold Kryptonite: Gold Kryptonite permanently removes a Kryptonian's superhuman abilities. Once expos

In [15]:
response.usage

CompletionUsage(completion_tokens=1107, prompt_tokens=17, total_tokens=1124)

# Temperature

In [16]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=1.2
)

CPU times: user 5.63 ms, sys: 1.36 ms, total: 6.99 ms
Wall time: 8.56 s


In [17]:
print(response.choices[0].message.content)

Once upon a time, in a small village nestled amidst lush green fields, there lived a young girl named Lily. Although she was only ten years old, Lily possessed an insatiable curiosity and boundless imagination.

One summer evening, as the warm sun began to set, Lily set off on her adventure to a nearby forest that was rumored to be enchanted. With her loyal companion, a mischievous ginger cat named Oliver, by her side, she was determined to uncover the secrets hidden within the mystical woods.

As they entered, a sense of mystery filled the air. Ancient trees, so tall that their branches seemed to touch the sky, towered above them. The forest floor was carpeted with delicate wildflowers, whispering enchanting melodies as the wind blew through them. The sunlight pierced through the thick foliage, casting intricate patterns of dancing shadows all around.

Lily's heart raced with anticipation as she followed Oliver, who seemed to be leading her deeper into the enchanted forest. The duo so

In [18]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=0
)

CPU times: user 5.67 ms, sys: 1.33 ms, total: 7 ms
Wall time: 7.72 s


In [19]:
print(response.choices[0].message.content)

Once upon a time, in a small village nestled in the heart of a lush forest, there lived a young girl named Lily. She was known for her kind heart and adventurous spirit. One sunny morning, as she was exploring the woods, she stumbled upon a wounded bird with a broken wing.

Lily carefully picked up the bird and cradled it in her hands. She knew she had to help it, so she decided to take it home. With great care, she fashioned a small nest for the bird and gently placed it inside. Lily named the bird Ruby, after its vibrant red feathers.

Days turned into weeks, and Lily tirelessly tended to Ruby's needs. She fed her, kept her warm, and even sang to her when she felt lonely. Slowly but surely, Ruby's wing began to heal, and she regained her strength.

One day, as Lily was preparing to release Ruby back into the wild, a sudden storm swept through the village. The wind howled, and rain poured down in torrents. Lily worried that Ruby wouldn't survive the harsh weather, so she made a quick 

## Few-shot prompting

We can also provide several examples of mappings between input and output.

In [20]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

This sudden change in direction means we don't have enough time to complete the entire project for the client.


# Formatted output

In [21]:
%%time
userInput = "blueberry pancakes"

prompt = """return a recipe for %s.
        Provide your response as a JSON object with the following schema:
        {"dish": "%s", "ingredients": ["", "", ...],
        "instructions": ["", "", ... ]}""" % (userInput, userInput)

response = client.chat.completions.create(
          model = "gpt-3.5-turbo",
          messages = [
            { "role": "system", "content": "You are a helpful recipe assistant."},
            { "role": "user",   "content": prompt }
          ]
)

CPU times: user 6.97 ms, sys: 1.6 ms, total: 8.56 ms
Wall time: 4.11 s


In [22]:
json_output = response.choices[0].message.content

In [23]:
output = json.loads(json_output)

In [24]:
output["ingredients"]

['1 cup all-purpose flour',
 '2 tablespoons sugar',
 '2 teaspoons baking powder',
 '1/4 teaspoon salt',
 '1 cup milk',
 '1 large egg',
 '2 tablespoons melted butter',
 '1 teaspoon vanilla extract',
 '1 cup fresh blueberries',
 'Extra butter or oil for cooking']

In [25]:
output["instructions"]

['In a large bowl, whisk together the flour, sugar, baking powder, and salt.',
 'In a separate bowl, whisk together the milk, egg, melted butter, and vanilla extract.',
 'Pour the wet ingredients into the dry ingredients and stir until just combined.',
 'Gently fold in the blueberries.',
 'Heat a non-stick skillet or griddle over medium heat and lightly grease with butter or oil.',
 'Using a 1/4 cup measuring cup, pour batter onto the skillet for each pancake.',
 'Cook until bubbles form on the surface of the pancake and the edges look set, about 2-3 minutes.',
 'Flip the pancake and cook for an additional 1-2 minutes, or until golden brown.',
 'Transfer the cooked pancakes to a plate and cover with a kitchen towel to keep warm.',
 'Repeat the process with the remaining batter, adding more butter or oil to the skillet as needed.',
 'Serve the blueberry pancakes warm with maple syrup and additional blueberries, if desired.']

# Translation

In [26]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": "You're a professional English-Italian translator."}, 
              {"role": "user", "content": "Translate 'Be the change that you wish to see in the world.' into Italian"}],
    temperature=0,
)

In [27]:
response.choices[0].message.content

'"Sii il cambiamento che desideri vedere nel mondo."'

# Process unstructured information

Inspired by https://platform.openai.com/examples/default-parse-data

In [28]:
prompt = """There are many fruits that were found on the recently discovered planet Goocrux. 
There are neoskizzles that grow there, which are purple and taste like candy. There are also 
loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits 
are a bright green color and are more savory than sweet. There are also plenty of loopnovas which 
are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which 
have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."""

In [29]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", 
         "content": "You will be provided with unstructured data, and your task is to parse it into CSV format."}, 
        {"role": "user", 
         "content": prompt}],
    temperature=0,
)

In [30]:
print(response.choices[0].message.content)

Fruit,Color,Taste
neoskizzles,purple,candy
loheckles,grayish blue,tart
pounits,bright green,savory
loopnovas,neon pink,cotton candy
glowls,pale orange,sour and bitter


In [31]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": """
            Read this paragraph 
            
            `%s` 
            
            and use it to answer some questions.""" % prompt}, 
              {"role": "user", "content": "What are pounits?"}],
    temperature=0,
)

In [32]:
print(response.choices[0].message.content)

Pounits are bright green fruits found on the recently discovered planet Goocrux. They have a more savory taste rather than being sweet.


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>